In [2]:
from collections import Counter
import json, random

with open("all_finegrained_clean.json", "r") as f:
    data = json.load(f)

edit_type_counts = Counter()
for d in data:
    for edit in d["fine_grained_edits"]:
        edit["categorization"] = edit["categorization"].replace("/ ", "/").replace(" (Unnecessary ornamental and overly verbose)", "")
        edit_type_counts[edit["categorization"]] += 1

final_categories = [cat for cat, count in edit_type_counts.most_common() if count >= 7]
data_validation = [d for d in data if d["split"] == "validation"]

print(len(data))
print(Counter([d["split"] for d in data]))
print(final_categories)

1057
Counter({'test': 911, 'validation': 146})
['Awkward Word Choice and Phrasing', 'Poor Sentence Structure', 'Unnecessary/Redundant Exposition', 'Cliche', 'Lack of Specificity and Detail', 'Purple Prose', 'Tense Inconsistency', 'Punctuation']


# 1. Create the Detection Prompt

## V1

In [10]:
def generate_detection_annotation(paragraph, problematic_span, words_before=6, words_after=6):
    start = paragraph.find(problematic_span)
    if start == -1 or paragraph.count(problematic_span) > 1:
        return None
    end = start + len(problematic_span)
    words_before_matched = 0

    while start > 0 and words_before_matched < words_before:
        if paragraph[start-1] == " ":
            words_before_matched += 1
        start -= 1
    words_after_matched = 0
    while end < len(paragraph) and words_after_matched < words_after:
        if paragraph[end] == " ":
            words_after_matched += 1
        end += 1

    padded_line = paragraph[start:end]
    padded_line = padded_line.replace(problematic_span, f"**{problematic_span}**")
    return padded_line

for NUM_FEWSHOTS in [5, 25]:
    prompt_detection = f"""You are given a paragraph of writing, and your goal is to provide feedback by selecting spans of text in the writing that could be improved, and assign each problematic span to an error category. Below, we list the {len(final_categories)} error categories that you can choose from, and provide a maximum of {NUM_FEWSHOTS} example problematic spans in text for that category. In each example, the problematic span is highlighted using ** at the start and end."""

    for i, category in enumerate(final_categories):
        prompt_detection += f"\n\nCategory {i+1}: '{category}':\n"
        examples = [(d, edit) for d in data_validation for edit in d["fine_grained_edits"] if edit["categorization"] == category]
        random.shuffle(examples)
        num_selected = 0
        for d, example in examples[:NUM_FEWSHOTS]:
            paragraph = d["preedit"]
            problematic_span = example["originalText"]
            detection_line = generate_detection_annotation(paragraph, problematic_span)
            if detection_line is None:
                continue
            prompt_detection += f"   Example {num_selected+1}: {detection_line}\n"

            num_selected += 1

    categories_STR = "|".join(final_categories)

    format_STR = """{"problematic_spans":
[
    {"span": "...", "category": "[[CATEGORIES]]"},
    ...
]
}""".replace("[[CATEGORIES]]", categories_STR)

    prompt_detection += f"""You must now provide feedback on the paragraph given below. Your feedback should follow the following JSON format:
{format_STR}

Rules:
- [Number of Spans] You can provide feedback on multiple spans, and multiple spans can have the same category.
- [Span must be verbatim] The span you select must be verbatim from the paragraph, otherwise, the feedback will not be provided to the user.
- [No Overlap] Spans should not overlap, and one span should not include the other.
- [Single Category] Each span should have exactly one category from the {len(final_categories)} categories listed above.
- [Diverse Edit Categories] Try to select spans that cover a diverse set of categories.

Paragraph:
[[PARAGRAPH]]"""

prompt_detection += '\n\nYour answer must start with `{"problematic_spans":`'

with open(f"prompts/detection_v1_fs{NUM_FEWSHOTS}.txt", "w") as f:
    f.write(prompt_detection)

## V2

In [3]:
import json, random

for NUM_FEWSHOTS in [2]:
    prompt_detection = f"""You are given a paragraph of writing, and your goal is to provide feedback by selecting spans of text in the writing that could be improved, and assign each problematic span to an error category. Below, we list the 7 error categories that you can choose from.

You are also provided {NUM_FEWSHOTS} examples of paragraphs that were annotated by professional writers, which you can use to better understand the task and the error categories.

Error Categories:
- "Awkward Word Choice and Phrasing": Suggestions for better word choices or more precise phrasing to enhance clarity and readability.
- "Cliche": The use of hackneyed phrases or overly common imagery that lack originality or depth.
- "Poor Sentence Structure": Feedback on the construction of sentences, recommending changes for better flow, clarity, or impact.
- "Unnecessary/Redundant Exposition": Redundant or non-essential parts of the text that could be removed/rephrased for conciseness.
- "Lack of Specificity and Detail": Need for more concrete details or specific information to enrich the text and make it more engaging.
- "Purple Prose": Identifying parts of the text that are seen as unnecessary ornamental and overly verbose.
- "Tense Consistency": Comments pointing out inconsistencies in verb tense that need to be addressed for uniformity.

Few-shot Examples:
"""

    random.shuffle(data_validation)

    for i, d in enumerate(data_validation[:NUM_FEWSHOTS]):

        input_text = d["preedit"]
        output = {"problematic_spans": [{"span": edit["originalText"], "category": edit["categorization"]} for edit in d["fine_grained_edits"]]}
        output_str = json.dumps(output)

        prompt_detection += f"\n\nExample {i+1}:\nInput Text:\n{input_text}\n\nOutput:\n{output_str}\n"

    prompt_detection += """You must now provide feedback on the paragraph given below. Your feedback should follow the JSON format provided in the examples above.

Rules:
- [Number of Spans] You can provide feedback on multiple spans, and multiple spans can have the same category.
- [Span must be verbatim] The span you select must be verbatim from the paragraph, otherwise, the feedback will not be provided to the user.
- [No Overlap] Spans should not overlap, and one span should not include the other.
- [Single Category] Each span should have exactly one category from the categories listed above.

Paragraph:
[[PARAGRAPH]]

Your answer must start with `{"problematic_spans":`
"""

    # print(prompt_detection)
    with open(f"prompts/detection_v2_fs{NUM_FEWSHOTS}.txt", "w") as f:
        f.write(prompt_detection)

# 2. Create the Suggestion Prompt

In [14]:
# create one prompt for each category
NUM_FEWSHOTS = 20

prompt_category = """You are given a paragraph of writing, and spans that have been identified as problematic, according the the category '[[CATEGORY]]'. Your goal is to propose a revision for each span that corrects the issue.

You should take inspiration from the following [[NUM_FEWSHOTS]] examples of problematic spans, and how they were each revised.

Examples:
[[EXAMPLES]]

Now perform the task for the paragraph given below, and the spans that have been identified as problematic.

Paragraph:
[[PARAGRAPH]]

Identified Spans:
[[SPANS]]

Rules:
- [Format] You should output your revision for each span in the following JSON format:
{"revisions":
    [
        {"span_id": "...", "revision": "..."},
        ...
    ]
}
- [Span ID] The span_id should match the span_id provided in the "Identified Spans" section.
- [Single Revision] You must provide one revision suggestion for each span in the "Identified Spans" section.
- [Paragraph Appropriateness] Your revision should be appropriate and fit within the paragraph given."""

for category in final_categories:
    prompt_revise_category = prompt_category.replace("[[CATEGORY]]", category).replace("[[NUM_FEWSHOTS]]", str(NUM_FEWSHOTS))
    examples_STR = ""
    examples = [(d, edit) for d in data for edit in d["fine_grained_edits"] if edit["final_categorization"] == category]
    
    random.shuffle(examples)

    for i, (d, example) in enumerate(examples[:NUM_FEWSHOTS]):
        paragraph = d["preedit"]
        context = generate_detection_annotation(paragraph, example["originalText"])
        if context is None:
            continue
        problematic_span = example["originalText"]
        revised_span = example["editedText"]
        examples_STR += f"Example {i+1}:\n"""
        examples_STR += 'Context: '+context+'\n'
        examples_STR += 'Input: {"span_id": '+str(i+1)+', "span": "'+problematic_span+'"}\n'
        examples_STR += 'Output: {"span_id": '+str(i+1)+', "revision": "'+revised_span+'"}\n\n'

    prompt_revise_category = prompt_revise_category.replace("[[EXAMPLES]]", examples_STR)

    # print("===================")
    # print(prompt_revise_category)
    clean_cat = category.replace(' ', '_').replace("/", "_")
    with open(f"prompts/revision_{clean_cat}_v1.txt", "w") as f:
        f.write(prompt_revise_category)

# 3. Run the System

In [2]:
from utils_generate_edits import run_edit_generation, build_revised_paragraph

# llm_engine = "claude3.5-sonnet"
# initial_paragraph = """She steps out into the crisp evening air, cigarette in hand. The first drag fills her lungs, menthol mingling with the damp chill. Tendrils of smoke curl around her as her thoughts drift to Shirley, like they always seem to lately. Memories surface - Shirley's crooked smile, her wild hair, the way her eyes crinkled when she laughed. But the good memories are fleeting, replaced by their last fight, the biting words exchanged. She can still see the hurt and anger in Shirley's eyes. The cigarette has burned down to the filter now. She stubs it out and lights another, hoping the nicotine will steady her shaking hands. Standing there, wreathed in smoke and regret, she wonders if they can ever go back to how things were. If too much has been said to salvage the fraying threads of their friendship. The thought makes her chest tighten. Crushing out the cigarette, she shoves her hands in her pockets and heads back inside, Shirley's absence an almost physical ache."""
initial_paragraph = """Eli jolted awake, gasping and drenched in sweat, the echoes of the nightmare still gnawing at his mind. The room, bathed in the soft gray light of dawn, felt stifling and confining, each shadow a potential harbinger of his dream's horrors. Desperate for an anchor, he stumbled to the window, pushing it open to let the brisk morning air clear the fog from his brain. Across the way, Mrs. Dwyer, his elderly neighbor, was watering her potted plants. Her movements were a quiet dance, precise and unhurried, each step deliberate. The way she tilted each vessel toward the light seemed almost sacred. Eli watched, mesmerized by her gentle rhythm, the simple act of nurturing life. His heartbeat gradually slowed, each breath syncing with the steady stream of water. He closed his eyes, allowing himself to be lulled by the intimacy of her routine. Before he knew it, the tension from his dream was a distant memory, replaced with a calm that felt both fragile and profound."""
# initial_paragraph = """Rodney, Leah, and Natalie sat around the kitchen table, each nursing a mug of coffee that had long since gone cold. Rodney leaned back in his chair, his eyes fixed on the ceiling as he listened to Leah's impassioned plea. She spoke of the environmental impact, the disruption to their quiet community, and the potential for increased traffic and noise. Natalie, ever the pragmatist, interjected with questions about the economic benefits and the possibility of improved infrastructure. Rodney remained silent, his thoughts torn between his love for the untouched beauty of their surroundings and the understanding that progress was inevitable. As the discussion grew more heated, Leah's voice took on a sharp edge, her frustration palpable in the way she gripped her mug. Natalie, unperturbed, continued to play devil's advocate, challenging Leah's arguments with a cool, measured tone. Rodney finally spoke, his voice soft but firm, suggesting a compromise—a way to work with the government to minimize the impact while still allowing for growth. Leah and Natalie exchanged glances, their expressions a mix of surprise and contemplation. The tension in the room eased slightly as they considered Rodney's words, each realizing that finding a middle ground might be the only way forward."""

for llm_engine in ["gpt-4o", "claude3.5-sonnet", "gemini-1.5-pro"]:
    print(f"-------- LLM Engine: {llm_engine} --------")
    edits = run_edit_generation(initial_paragraph, llm_engine=llm_engine, printing=False)

    revised_paragraph, revised_paragraph_diff = build_revised_paragraph(initial_paragraph, edits)
    
    print(revised_paragraph_diff)

-------- LLM Engine: gpt-4o --------
 Eli jolted awake, gasping and drenched in sweat, theechoesremnants of the nightmare stillgnawing atlingering in hismind.thoughts [Cliche]. The room,bathed insoftly illuminated by thesoft grayearly morning lightof dawn,[Cliche], feltstiflingoppressive [Word Choice andconfining,Phrasing], each shadow apotential harbingerreminder ofhis dream's horrors.the nightmare [Purple Prose]. Desperate for an anchor, he stumbled to the window, pushing it open to let the brisk morning air clear the fog from his brain. Across the way, Mrs. Dwyer, his elderly neighbor, was watering her potted plants. Her movements werea quiet dance, precisemoving slowly andunhurried, each step deliberate.purposefully [Purple Prose]. The way she tilted each vessel toward the lightseemed almost sacred.looked graceful [Purple Prose]. Eli watched, mesmerized by her gentle rhythm, thesimple act of nurturing life.deliberate care she took with each plant [Cliche]. His heartbeat gradually s